In [5]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--jars lib/graphframes-release-0-5-0-assembly-0.5.0-spark2.1.jar pyspark-shell')
from pyspark.sql import SparkSession
from lib.graphframes import *
from pyspark.sql.functions import col, size, lit, collect_list

spark = SparkSession \
    .builder \
    .appName("RelaTree") \
    .getOrCreate()

# group_members_data = spark.read.csv("data/group_members.csv", header="true").drop("update_time", "is_admin")
# group_channel_data = spark.read.csv("data/group_channel.csv", header="true")

In [6]:
# Vertex DataFrame
v = spark.createDataFrame([
    ("a", "Alice", 34),
    ("b", "Bob", 36),
    ("c", "Charlie", 30),
    ("d", "David", 29),
    ("e", "Esther", 32),
    ("f", "Fanny", 36),
    ("g", "Gabby", 60)
], ["id", "name", "age"])
# Edge DataFrame
e = spark.createDataFrame([
    ("a", "b", "friend"),
    ("b", "c", "follow"),
    ("c", "b", "follow"),
    ("f", "c", "follow"),
    ("e", "f", "follow"),
    ("e", "d", "friend"),
    ("d", "a", "friend"),
    ("a", "e", "friend")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)
g.vertices.show()
g.edges.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
+---+---+------------+



In [6]:
g.vertices.write.parquet('store/gv.parquet')
g.edges.write.parquet('store/ge.parquet')

agv = spark.read.parquet('store/gv.parquet')
age = spark.read.parquet('store/ge.parquet')

ag = GraphFrame(agv, age)
ag.vertices.show()
ag.edges.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  c|Charlie| 30|
|  e| Esther| 32|
|  a|  Alice| 34|
|  d|  David| 29|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
|  b|    Bob| 36|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
+---+---+------------+



In [18]:
import pandas as pd

data = pd.read_csv('data/group_members.csv')
data.head()

c:\programdata\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,create_time,update_time,group_id,user_id,is_admin,added_by
0,2016-06-25 12:12:58.776567,2016-06-25 12:12:58.776567,c8a5ca5947cc7d9251b808b7aad0e75fac381823,fcaa201f972dfc42809e83de776fd36aa1577761,f,NaN
1,2018-03-26 18:30:02.531757,2018-03-26 18:30:02.531757,d2c8410bb78af46155aaa96b50b082598ca69306,26cf4a8044866c1bca5f54dea9632180ac37e6d6,f,NaN
2,2017-12-18 10:23:26.487894,2017-12-18 10:23:26.487894,2c07f080d9f041295761260edf28275a872e65e2,570a2ddfdccc2b7754dfc1052923217a8e5fd2df,f,NaN
3,2018-02-08 16:58:31.057972,2018-02-08 16:58:31.057972,1430c9033779d7585789131f4d287743c9163e87,df8d3021822ef6fed29011bc254c5d82d99222dd,f,NaN
4,2018-03-26 18:30:02.531757,2018-03-26 18:30:02.531757,d2c8410bb78af46155aaa96b50b082598ca69306,777862a1db383bcd7ea66d9f80424f98a7c9ef9f,f,NaN
